In [1]:
import Bio

In [51]:
import pandas as pd
import numpy as np

Read the `Resfams.tbl` file to drop lines where the first column is longer than 20 characters.  Also skip any rows that begin with a `#`.  Save the output to `Resfams.clean.tbl`.  Rejoice!

In [69]:
with open('Resfams.tbl', 'rt') as input, open('Resfams.clean.tbl', 'wt') as output, open('Resfams.dirty.tbl', 'wt') as dirty:
    for line in input:
        fields = line.split()
        if fields:
            if fields[0][0] != '#':
                if len(fields[0]) < 21:
                    output.write(line);
                else:
                    dirty.write(line);
            else:
                pass

Read the modified table using the fixed width reader.  Note we manually specify the dtypes and column names.  Bummer.

In [70]:
df = pd.read_fwf('Resfams.clean.tbl', widths=[21, 11, 21, 11, 10, 7, 6, 10, 7, 6, 4, 4, 4, 4, 4, 4, 4, 4, 1024], 
    header=None, error_bad_lines=False, warn_bad_lines=True,
    names=[ "target", "target_accession", "query", "query_accesion", "full_evalue", "full_score", 
           "full_bias", "best_evalue", "best_score", "best_bias", "exp", "reg", "clu", "ov", "env", 
           "dom", "rep", "inc", "description" ],
    dtype={
        "target" : object, 
        "target_accession" : object, 
        "query" : object, 
        "query_accesion" : object, 
        "full_evalue" : np.float64, 
        "full_score" : np.float64, 
        "full_bias" : np.float64, 
        "best_evalue" : np.float64, 
        "best_score" : np.float64, 
        "best_bias" : np.float64, 
        "exp" : np.float64, 
        "reg" : np.float64, 
        "clu" : np.int64, 
        "ov" : np.int64, 
        "env" : np.int64, 
        "dom" : np.int64, 
        "rep" : np.int64, 
        "inc" : np.int64, 
        "description" : object, 
    })
df.head()

ValueError: Unable to convert column full_evalue to type <class 'numpy.float64'>

In [53]:
df.groupby("query")["full_evalue"].min()

query
NP_037570.1      0.0033
NP_037571.1     0.00063
NP_037574.1     1.1e-26
NP_037576.1     1.8e-21
NP_040407.1      1.2e-1
NP_040409.1     6.5e-05
NP_040422.1     1.3e-48
NP_040433.1     3.2e-80
NP_040437.1     9.9e-87
NP_040462.1       1.2e-
NP_040465.1    1.1e-104
NP_040470.1     2.7e-64
NP_040471.1     3.4e-33
NP_040475.1      2.2e-1
NP_043524.1     1.3e-48
NP_044260.1     6.6e-37
NP_044301.1     1.2e-43
NP_044302.1    2.4e-150
NP_044360.1     2.7e-64
NP_044444.1     1.1e-12
NP_044466.1     1.9e-90
NP_045488.1     0.00066
NP_045557.1       0.012
NP_045650.1     1.4e-39
NP_045651.2     2.1e-07
NP_046407.1     0.00014
NP_047310.1     0.00019
NP_047312.1     1.1e-05
NP_047316.1     1.2e-20
NP_047322.1     2.4e-24
                 ...   
YP_783929.1     0.00023
YP_783930.1     2.2e-18
YP_787997.1     1.2e-08
YP_788000.1       1e-28
YP_788044.1         1e-
YP_788126.1     1.8e-41
YP_788128.1    3.1e-115
YP_794147.1        2e-2
YP_796434.1     1.5e-09
YP_796443.1     1.1e-45
YP_891141.

In [57]:
df.dtypes

target               object
target_accession     object
query                object
query_accesion       object
full_evalue          object
full_score           object
full_bias            object
best_evalue          object
best_score           object
best_bias            object
exp                 float64
reg                 float64
clu                   int64
ov                    int64
env                   int64
dom                   int64
rep                   int64
inc                   int64
description          object
dtype: object